In [ ]:
import numpy as np
import pandas as pd
# df=pd.read_excel("allfeatures.xlsx")
df=pd.read_csv("allfeatures.csv")
print(f'shapte of feateures: {df.shape}')
print(f'total na values: {df.isna().sum().sum()}')
print(f'total null values: {df.isnull().sum().sum()}')

shapte of feateures: (7722, 1389)
total na values: 0
total null values: 0


In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming df is your DataFrame and it's already loaded
X = df.drop(['pcid', 'class', 'smile'], axis=1)  # feature matrix
y = df['class']  # label vector

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model definition with increased max_iter
model = LogisticRegression(solver='lbfgs', random_state=42, max_iter=10000)

# Parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best AUC: {:.4f}".format(grid_search.best_score_))

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:,1]

# Calculate class predictions with a default threshold of 0.5
y_pred = (y_pred_proba >= 0.5).astype(int)

# Detailed classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

# Individual metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)  # AUC using predicted probabilities
accuracy = accuracy_score(y_test, y_pred)

# Print individual metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Accuracy: {accuracy:.2f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters: {'C': 0.01}
Best AUC: 0.9296
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       807
           1       0.87      0.82      0.85       738

    accuracy                           0.86      1545
   macro avg       0.86      0.86      0.86      1545
weighted avg       0.86      0.86      0.86      1545

Precision: 0.8746
Recall: 0.8225
F1 Score: 0.8478
Test AUC: 0.9326
